In [ ]:
from qat.lang.AQASM import Program, H, CNOT, RZ,  AbstractGate
import numpy as np
from helpers import ansatz, ham_simulation

In [ ]:
nBits = 8

import json
import matplotlib.pylab as plt

with open('hamiltonian_data.json','r') as f:
    hams = json.load(f)

In [ ]:
coeffs = hams[0]

In [ ]:
import numpy as np

I = np.eye(4)
#plt.matshow(I)

Z0 = np.diag([1,1,-1,-1])
#plt.matshow(Z0)

Z1 = np.diag([1,-1,1,-1])
#plt.matshow(Z1)

Z0Z1 = np.diag([1,-1,-1,1])
#plt.matshow(Z0Z1)

X0X1 = np.fliplr(np.eye(4))
#plt.matshow(X0X1)

Y0Y1 = np.fliplr(np.diag([-1, 1, 1, -1]))
#plt.matshow(Y0Y1)

In [ ]:
shift = 2.

Ham = coeffs['I_coeff'] * I
Ham += coeffs['Z0_coeff'] * Z0
Ham += coeffs['Z1_coeff'] * Z1
Ham += coeffs['Z0Z1_coeff'] * Z0Z1
Ham += coeffs['X0X1_coeff'] * X0X1
Ham += coeffs['Y0Y1_coeff'] * Y0Y1
Ham += shift * I
w, v = np.linalg.eigh(Ham)

In [ ]:
def eigen_vecs(coeffs, shift):
    Ham = coeffs['I_coeff'] * I
    Ham += coeffs['Z0_coeff'] * Z0
    Ham += coeffs['Z1_coeff'] * Z1
    Ham += coeffs['Z0Z1_coeff'] * Z0Z1
    Ham += coeffs['X0X1_coeff'] * X0X1
    Ham += coeffs['Y0Y1_coeff'] * Y0Y1
    Ham += shift * I
    w, v = np.linalg.eigh(Ham)
    return v

eigen_prep = AbstractGate("eigen", 
                          [dict, float], 
                          arity = 2, 
                          matrix_generator=eigen_vecs)

In [ ]:
prog = Program()
prog_ansatz = Program()

q = prog.qalloc(3)
q2 = prog_ansatz.qalloc(3)

prog.apply(eigen_prep(coeffs, shift), q[1], q[2])
prog_ansatz.apply(ansatz(0.), q[1], q[2])

circ = prog.to_circ()
circ_ansatz = prog_ansatz.to_circ()

from qat.qpus import get_default_qpu
qpu = get_default_qpu()

In [ ]:
job = circ.to_job(qubits = [0,1,2])
job_ansatz = circ_ansatz.to_job()

result = qpu.submit(job)
result_ansatz = qpu.submit(job_ansatz)

print("Ground state:")
for sample in result:
    print(sample.probability,sample.amplitude,sample.state)
    
print("Ansatz:")
for sample in result_ansatz:
    print(sample.probability,sample.amplitude,sample.state)

In [ ]:
print(v[:,0])


In [ ]:
print(v,w)

In [ ]:
from scipy import linalg

E_max = 3
E_min = 0.5
    
dt = (2 * np.pi) / float(E_max)
    

def unit_ev(coeffs, shift, k):
    Ham = coeffs['I_coeff'] * I
    Ham += coeffs['Z0_coeff'] * Z0
    Ham += coeffs['Z1_coeff'] * Z1
    Ham += coeffs['Z0Z1_coeff'] * Z0Z1
    Ham += coeffs['X0X1_coeff'] * X0X1
    Ham += coeffs['Y0Y1_coeff'] * Y0Y1
    Ham += shift * I

    return linalg.expm(-1j * Ham * (2**k) * dt)

u_k = AbstractGate("uk", 
                   [dict, float, int], 
                   arity=2, 
                   matrix_generator=unit_ev)

In [ ]:
bits = {}

for k in range(nBits, 0, -1):
    prog = Program()
    
    q = prog.qalloc(3)
    
    prog.apply(H, q[0])    
    prog.apply(eigen_prep(coeffs, shift), q[1], q[2])

    prog.apply(u_k(coeffs, shift, k-1).ctrl(), q)
    
    phi_k = 0

    for l in range(k+1, nBits+1, 1):
        phi_k += bits[l] / float(2**(l-k+1))
    
    phi_k *= 2 * np.pi
    
    prog.apply(RZ(phi_k), q[0])
    prog.apply(H, q[0])
    
    job = prog.to_circ().to_job(qubits=[0])
    
    result = qpu.submit(job)
    
    max_proba = -1
    max_state = -1
    for sample in result:
        print(sample.probability, sample.state)
        if sample.probability > max_proba:
            max_proba = sample.probability
            max_state = sample._state
     
    bits[k] = max_state
    
print(bits)
print([bits[k] for k in range(1,nBits+1,1)])

In [ ]:
phi = 0
for l in range(1,nBits+1):
    print(l)
    phi += float(bits[l])/float(2**l)
    
print(phi)

In [ ]:
def phase(coeffs, ansatz_prep=False, ansatz_angle=0., trotterization=False, trotter_number=4):
    
    bits = {}

    for k in range(nBits, 0, -1):
        prog = Program()

        q = prog.qalloc(3)

        prog.apply(H, q[0])
        if ansatz_prep:
            prog.apply(ansatz(ansatz_angle), q[1], q[2])
        else:
            prog.apply(eigen_prep(coeffs, shift), q[1], q[2])

        if trotterization:
            prog.apply(ham_simulation(coeffs, (2**(k-1)) * dt, trotter_number, shift).ctrl(), q)
        else:
            prog.apply(u_k(coeffs, shift, k-1).ctrl(), q)

        phi_k = 0

        for l in range(k+1, nBits+1, 1):
            phi_k += bits[l] / float(2**(l-k+1))

        phi_k *= 2 * np.pi

        prog.apply(RZ(phi_k), q[0])
        prog.apply(H, q[0])

        job = prog.to_circ().to_job(qubits=[0])

        result = qpu.submit(job)

        max_proba = -1
        max_state = -1
        for sample in result:
            if sample.probability > max_proba:
                max_proba = sample.probability
                max_state = sample._state
     
        bits[k] = max_state
    
#    print([bits[k] for k in range(1,nBits+1,1)])
        
    phi = 0
    for l in range(1,nBits+1,1):
        phi += float(bits[l])/float(2**l)
            
    return phi


In [ ]:
vals = []
vals_ansatz = []
vals_trotter = []
vals_all = []
Rs = []
for coeffs in hams:
    phi_c = phase(coeffs)
    phi_ansatz = phase(coeffs, ansatz_prep=True, ansatz_angle=0.)
    phi_trotter = phase(coeffs, trotterization=True, trotter_number=100)
    phi_all = phase(coeffs, trotterization = True, ansatz_prep=True, trotter_number=10)
    
    E = ((2*np.pi)/dt)*phi_c
    E_ansatz = ((2*np.pi)/dt)*phi_ansatz
    E_trotter = ((2*np.pi)/dt)*phi_trotter
    E_all = ((2*np.pi)/dt)*phi_all
    
    print("R", coeffs['R'])
    Rs.append(coeffs['R'])
    
    vals.append(E-shift)
    vals_ansatz.append(E_ansatz-shift)
    vals_trotter.append(E_trotter-shift)
    vals_all.append(E_all-shift)


In [ ]:
plt.plot(Rs,vals, label="perfect")
plt.plot(Rs,vals_ansatz, label="perfect+ansatz")
plt.plot(Rs, vals_trotter, label="perfect+trotter")
plt.plot(Rs, vals_all, label="all")
plt.legend()

In [ ]:
from helpers import u_yy_matrix

print(u_yy_matrix(0.3))

print(linalg.expm(-1j*Y0Y1*0.3))

print("SUM of DIFF ",(u_yy_matrix(0.3)-linalg.expm(-1j*Y0Y1*0.3)).sum())

In [ ]:
import numpy as np

H = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])
S = np.diag([1,1j])
Sd = np.diag([1,-1j])

Y = np.array([[0,-1j],[1j, 0]])

Z = np.diag([1,-1])

W = np.dot(S,np.dot(H,S))
Wd = np.dot(Sd, np.dot(H,Sd))


print(np.dot(Wd,np.dot(Y,W)))